# TensorFlow入門2 ロジスティック回帰実装

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf
tf.__version__

'1.14.0'

## 1.このテキストについて

### テキストの目的
- TensorFlowで学習を行う

### どのように学ぶか
サンプルコードとともに説明していきます。同じコードを打ち込んで実行していってください。

## 2.ロジスティック回帰の実装

TensorFlowを使いロジスティック回帰を実装していきます。入門1では単純な足し算でしたが、ここでは学習を伴う計算を行います。

### 論理回路
簡単な題材として、ロジスティック回帰による論理回路の再現を行います。論理回路は2つの値を入力し、1つの値を出力する関数のようなものです。入力も出力も0か1のみで表され、入力される組み合わせによって出力する値が変わります。

ANDゲートは入力された2つの値が両方とも1だった場合、出力が1となり、それ以外の組み合わせでは0を出力します。

| 入力1 | 入力2 | 出力 |
| --- | --- | --- |
| 0	| 0	| 0 |
| 1 | 0 | 0 |
| 0 | 1 | 0 |
| 1 | 1 | 1 |

![image](https://t.gyazo.com/teams/diveintocode/24b9632f52dcdd8c226a1be3eccc2361.png)

これを学習させます。

### データの作成
最初に学習用のトレーニングデータをNumPyにより作成しておきます。ANDゲートでは入力が2次元で出力が1次元となります。2つの入力が1のときだけ1を出力し、それ以外は0を出力するので以下のように定義できます。

In [3]:
import numpy as np
x_train = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_train = np.array([[0], [0], [0], [1]])

TensorFlowによる実装に必要な次の2つのことを順番に見ていきます。

1. データフローグラフを構築する
2. データを入力して計算する

### データフローグラフの構築

まずはデータフローグラフを構築します。

学習データをTensorFlowのデータフローグラフに入力するための **placeholder** を用意しましょう。placeholderはデータフローグラフを作成する段階では値が決まっていない、空箱のような存在でした。

In [4]:
x = tf.placeholder(tf.float32, [None, 2])
t = tf.placeholder(tf.float32, [None, 1])

第一引数の`tf.float32`で行列要素の数値のデータ型を指定しています。第二引数の`[None,2]`で行列の形を指定しています。ここで定義されている`2`はデータの次元を表しています。Noneの部分はデータ数を表す部分です。今回のANDゲートの場合のデータ数は[0,0],[0,1],[1,0],[1,1]の4つしかないのでNoneの部分を`[4,2]`としても問題はありません。しかし、任意の数のデータを入れられるように、一般的にはNoneを使います。

[tf.placeholder  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/compat/v1/placeholder?authuser=0&hl=ja)

重みとバイアスの **Valiable** を用意します。Valiableとして用意するということは、これらが学習により更新を行う値であることを示します。

In [5]:
W = tf.Variable(tf.zeros([2,1]))
b = tf.Variable(tf.zeros([1]))

ここで、`tf.Variable()`の中で`tf.zeros()`という関数を呼び出していますが、初期値として0を入れているということです。

[tf.Variable  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/Variable)

[tf.zeros  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/zeros)

次にモデルの出力y（＝仮定関数）と目的関数を定義します。

ロジスティック回帰の式は以下でした。ここでは正則化項は抜かしています。
$$
h_θ(x) = g(θ^T x).\\ g(z) = \frac{1}{1+e^{−z}}.\\ J(\theta)=  \frac{1}{m}  \sum_{i=1}^{m}[−y^{(i)} log(h_θ(x^{(i)})) − (1−y^{(i)}) log(1−h_θ(x^{(i)}))]
$$

$m$ : 入力されるデータの数

$h_\theta()$ : 仮定関数

$x$ : 特徴量ベクトル

$\theta$ : パラメータベクトル

$g()$ : シグモイド関数

$x^{(i)}$ : i番目のサンプルの特徴量ベクトル

$y^{(i)}$ : i番目のサンプルの正解ラベル

$\theta_j$ : j番目のパラメータ（重み）

$n$ : 特徴量の数


これをTensorFlowで記述すると次のようになります。

In [6]:
y = tf.sigmoid(tf.matmul(x, W) + b)
cross_entropy = tf.reduce_sum(-t * tf.log(y) - (1 - t) * tf.log(1 - y))

`tf.matmul()`はNumPyにおける`np.dot()`に相当するベクトルの内積や、行列積を計算するためのメソッドです。

[tf.math.sigmoid  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/sigmoid)

[tf.math.log  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/math/log?hl=ja)

[tf.math.reduce_sum  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/reduce_sum)

なお、例えば回帰問題で二乗和誤差関数を使用するのであれば、`tf.reduce_sum(tf.square(y - t))`というように定義できます。

[tf.math.square  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/square)

ここまでで、入力のための空箱であるplaceholderと学習可能なValiableをメソッドで結ぶことができました。

学習を行うために、勾配降下法を用いてパラメータを最適化するためのコードを加えます。目的関数をGradientDescentOptimizerに渡します。

In [7]:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

GradientDescentOptimizer()は引数で学習率を指定しています。

[tf.train.GradientDescentOptimizer  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/compat/v1/train/GradientDescentOptimizer)

学習後の結果の正解が正しいかどうかの判定と正解率の計算もデータフローグラフとして定義できます。

In [8]:
correct_prediction = tf.equal(tf.sign(y - 0.5), tf.sign(t - 0.5))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

1行目で結果が正解かどうか判定しています。一つ一つ見ていきましょう。まず`tf.equal()`は引数に指定された2つの値が等しいかどうかを判定してくれます。返り値はBool値です。`tf.sign()`は引数の値が正なら1、0なら0、負なら-1を返します。yが0.5以上かどうかで結果が決まるので、y-0.5とt-0.5の符号を比較しています。

2行目は正解率を計算するためのコードです。`tf.reduce_mean()`は多次元配列の各成分の平均を計算する関数です。`tf.cast()`でBool値を0,1に変換しています。つまりここでは正解で1、不正解で0と判定された配列の平均値をとっているので正解率を表していることになります。

[tf.math.equal  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/equal)

[tf.math.sign  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/sign)

[tf.math.reduce_mean  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/reduce_mean)

[tf.dtypes.cast  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/cast)

以上で学習のための準備は終わりです。データフローグラフをコードで定義できました。

### データを入力して計算

セッションを準備してパラメータを最適化する計算を行います。

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

まずセッションのインスタンスを作成します。そして、tf.global_variables_initializer()によって上で定義したtf.Variable()の値(重み・バイアス)を初期化します。実行する際にはsess.run()を使います。

[tf.initializers.global_variables  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/compat/v1/global_variables_initializer?hl=ja)

学習を行います。今回は1000回繰り返すことにします。

In [10]:
for epoch in range(1000):
    sess.run(train_step, feed_dict={
        x:x_train,
        t:y_train
    })
# 100回ごとに正解率を表示
    if epoch % 100 == 0:
        acc_val = sess.run(
            accuracy, feed_dict={
                x:x_train,
                t:y_train})
        print ('epoch: %d, Accuracy: %f'
               %(epoch, acc_val))

epoch: 0, Accuracy: 0.750000
epoch: 100, Accuracy: 1.000000
epoch: 200, Accuracy: 1.000000
epoch: 300, Accuracy: 1.000000
epoch: 400, Accuracy: 1.000000
epoch: 500, Accuracy: 1.000000
epoch: 600, Accuracy: 1.000000
epoch: 700, Accuracy: 1.000000
epoch: 800, Accuracy: 1.000000
epoch: 900, Accuracy: 1.000000


2行目、`sess.run()`に上で定義したtrain_stepを入れることで、勾配降下法による学習を行っています。

8行目は`sess.run()`にaccuracyを入れることで、正解率を計算しています。計算結果がNumPy形式で返ってきているので、これをprintします。

形だけ定義していたplaceholderのxとtの中には値が何も入っていません。placeholderに値を設定するために`sess.run()`のパラメータで`feed_dict`を指定します。例えば、`feed_dict={x:x_train,t:y_train})`と書くことで空箱だったxにx_trainの値が入り、tにy_trainの値が入ります。

表示結果を見てみると、正解率が100%でうまく学習できているように見えます。

最後に各サンプルの計算結果を確認してロジスティック回帰の実装を終わります。先ほどのaccuracyと同様、実行することで計算結果が返ってくるためprintします。

In [11]:
#学習結果が正しいか確認
classified = sess.run(correct_prediction, feed_dict={
    x:x_train,
    t:y_train
})
#出力yの確認
prob = sess.run(y, feed_dict={
    x:x_train,
    t:y_train
})
print(classified)
# [[ True]
# [ True]
# [ True]
# [ True]]
print(prob)
# [[  1.96514215e-04]
# [  4.90498319e-02]
# [  4.90498319e-02]
# [  9.31203783e-01]]

[[ True]
 [ True]
 [ True]
 [ True]]
[[1.9651403e-04]
 [4.9049813e-02]
 [4.9049813e-02]
 [9.3120378e-01]]


classifiedの結果は全てTrueで正しく学習されていることがわかります。probは上からほぼ[0,0,0,1]となっています。今回活性化関数に用いたのはシグモイド関数ですので出力yは確率として表示されています。上から3つは1になる確率がほぼ0％、一番下の1つは93％程度の確率で1になるということになります。

Wとbが学習後どのような値になっているかも見ておきましょう。Variableも`sess.run()`に入れることで値を確認できます。

In [12]:
print('W:', sess.run(W))
print('b:', sess.run(b))
# W: [[ 5.5699544]
# [ 5.5699544]]
# b: [-8.53457928]

W: [[5.569955]
 [5.569955]]
b: [-8.53458]


### 途中の値が見たい場合
デバッグのために途中の値が見たい場合もあります。例えば、`y = tf.sigmoid(tf.matmul(x, W) + b)`の`tf.matmul(x, W)`の部分が見たいといったことを考えます。

出力yが`sess.run(y, feed_dict={x:x_train, t:y_train})`で見れたことと同様の行えば良いため、次のようなコードになります。

In [13]:
mat = tf.matmul(x, W)
y = tf.sigmoid(mat + b)
print(sess.run(mat, feed_dict={
    x:x_train,
    t:y_train
}))

[[ 0.      ]
 [ 5.569955]
 [ 5.569955]
 [11.13991 ]]


データフローグラフを構築し、それを`sess.run()`で実行するという流れはここでも同じです。

### セッションの終了
最後にセッションは終了させます。

In [14]:
sess.close()

with構文を使うことも可能です。
```
with tf.Session() as sess:
    sess.run() # ここに計算の実行コードを入れていく
```

### NumPyによるスクラッチ実装との比較
TensorFlowを利用した場合、NumPyでのスクラッチのように更新の式ということを考える必要はなくなりました。データフローグラフの構築は、スクラッチにおけるフォワードプロパゲーションの実装に近いと言えます。また、シグモイド関数などよく使われるものは関数化されているため、それらを組み合わせていくだけで実装することが可能です。